In [5]:
import sqlite3
import json
import codecs

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    if not('status' in js and js['status'] == 'OK') : continue

    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    if lat == 0 or lng == 0 : continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'", "")
    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
print("Open where.html to view the data in a browser")



Dormitory no. 14 "Kapitol" of AGH University of Science and Technology in Cracov, Poland, Witolda Budryka 2, 30-072 Kraków, Poland 50.0676785 19.9070824
Warsaw, Poland 52.2296756 21.0122287
American University Housing District, Cairo Governorate, Egypt 30.0068972 31.5005988
State Ave, Tempe, AZ 85281, USA 33.4232051 -111.8879509
Information Technology Services, 2500 Daniels Bridge Rd, Athens, GA 30606, USA 33.9094658 -83.4604372
BITS, Pilani, Rajasthan 333031, India 28.3580163 75.5887989
Babcock Dr, Rochester, NY 14610, USA 43.1294077 -77.5479304
Banaras Hindu University Campus, Varanasi, Uttar Pradesh, India 25.2685304 82.9904737
Bangalore University Rd, Bengaluru, Karnataka 560056, India 12.9426914 77.5053145
Baylor Blvd, Fairbanks, AK 99709, USA 64.8852156 -147.8009743
Capital Normal University, Haidian, Beijing, China, 100037 39.9304437 116.3076925
Monument to Fallen Professors and Students of the Belarusian State University, Minsk, Belarus 53.89318369999999 27.547338
Belgrade Dr, 

In [3]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    serviceurl = "http://py4e-data.dr-chuck.net/geojson?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("where.data")
count = 0
for line in fh:
    if count > 20000 :
        print('Retrieved 200 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    parms = dict()
    parms["query"] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(0)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")



Found in database  AGH University of Science and Technology

Found in database  Academy of Fine Arts Warsaw Poland

Found in database  American University in Cairo

Found in database  Arizona State University

Found in database  Athens Information Technology

Found in database  BITS Pilani

Found in database  Babcock University

Found in database  Banaras Hindu University

Found in database  Bangalore University

Found in database  Baylor University

Found in database  Beijing normal university

Found in database  Belarusian State University

Found in database  Belgrade University

Found in database  Beloit College

Found in database  Belorussian State University

Found in database  Ben Gurion University

Found in database  Bharthidasan University

Found in database  Boston University

Found in database  California Polytechnic State University of San Luis Obispo

Found in database  California State University San Bernardino

Found in database  City of Westminster College

Found in dat

Retrieved 1938 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=Universite+Catholique+de+Louvain
Retrieved 1912 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+College+Dublin
Retrieved 11147 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+Munich
Retrieved 15191 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Akron
Retrieved 1965 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Alberta
Retrieved 2262 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Amsterdam
Retrieved 1951 characters {    "results" : [  
Pausing for a bit...

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Arkansas
Retrieved 2322 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/g

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Twente
Retrieved 2124 characters {    "results" : [  
Pausing for a bit...

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Utah
Retrieved 10547 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Vienna
Retrieved 2107 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Warsaw
Retrieved 1928 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Washington
Retrieved 2275 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Washington+-+Bothell
Retrieved 2004 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+Waterloo
Retrieved 2001 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/geojson?query=University+of+West+Florida
Retrieved 